**Building Dashboard in Python with Dash and Plotly Express**

Rita Miller, Data Scientist, www.linkedin.com/in/ rita-m-135953239

In [ ]:
#Save file in Google and then mount Google Drive account
#from google.colab import drive
#drive.mount('/content/drive')
#your Drive account will be mounted to the /content/drive directory in Colab.
#https://saturncloud.io/blog/how-to-read-a-file-from-drive-in-google-colab/

In [1]:
#Reads Google Sheet into Python, but first authenticate
from google.colab import auth
auth.authenticate_user()

In [2]:
#Imports the necessary libraries and initialize my credentials
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
#Open the desired sheet and get all values
#worksheet = gc.open('RRTOnly').get_worksheet(0) #Aug
worksheet = gc.open('RRTOnly').get_worksheet(1) #Sep
rows = worksheet.get_all_values()

In [4]:
#Import Python Libraries

#Data manipulation and analysis
import pandas as pd
#Fundamental package for scientific computing with Python
import numpy as np

##Data Visualization Packages
import matplotlib.pyplot as plt #fundamental plotting library in Python.
#The Seaborn library is used for statistical data visualization in Python.
#Seaborn is built on top of matplotlib and provides a high-level interface for drawing
#attractive and informative statistical graphics.
import seaborn as sns

#Dashboard
!pip install dash #dont comment out
import dash #Dash is a Python framework for building web applications
##Dash core components (dcc) contains the various building blocks to create the app
from dash import dcc, html
import plotly.express as px

#adds an invisible trace to the plot, which appears in the legend with the total count.
#so the total count is displayed in the legend without affecting the plot itself
import plotly.graph_objects as go

#provides a way to interact with the operating system
import os

#imports the date and datetime classes from the datetime module
from datetime import date, datetime

#Callbacks
from dash.dependencies import Input, Output

#Suppress all warnings and garbage collector
import warnings, gc
warnings.filterwarnings('ignore')

# Enable garbage collection
gc.enable()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 33.4 MB/s eta 0:00:00


In [5]:
#Convert to a pandas dataframe.
df = pd.DataFrame.from_records(rows[1:])
print(df.head())

        0      1               2    3     4  5     6
0  9/1/24  23:00     Respiratory  RRT   7SW  Y   ICU
1                                                   
2  9/1/24   7:18  Cardiovascular  RRT  10SW  N  Stay
3                                                   
4                                                   


In [6]:
# Replace whitespace characters with NaN
df_cleaned = df.replace(r'^\s*$', np.nan, regex=True)
df_cleaned = df_cleaned.dropna(how='all')
#print("Cleaned DataFrame shape:", df_cleaned.shape)

In [31]:
#Introduction

# Define the Google Drive link
#image_link = 'https://drive.google.com/file/d/1j2lggF3BPbVS9ksikYTe7GDmjJ_nl7AJ/view?usp=sharing'

#Image not showing, try

#Image not showing, path from Google drive verified
#image_link = '/content/drive/MyDrive/Tech_Projects/Amber_Depew_Stat_Team_Swedish/Assets/nurse.png'

#created folder named assets in same directory
#image_link = 'https://drive.google.com/file/d/1j2lggF3BPbVS9ksikYTe7GDmjJ_nl7AJ/view?usp=drive_link'

#Image not showing, try

#Converted Google Drive link to a direct link
#image_link = 'https://drive.google.com/drive/folders/1Sw-1b2Cl4TCC3dS0UU4RjUy6vcI6N_II?usp=drive_link'

#Image not showing, try

#Picture placed in Amazon pics
image_link = 'https://www.amazon.com/photos/shared/adIpO3nGRaC70MsE-CBtjQ.qaO7ZleuoKEI6qSuZGdaex'

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

app.layout = html.Div(
    children=[
        # Add created image
        #html.Img(src=image_link,
                 # Set the size of the image
                 #style={'width': '100%', 'height': 'auto'}),
        # Add header text
        html.H1('Swedish First Hill - RRT Dashboard', style={'color': 'black'}),
        # Add paragraph text
        html.P(children=[
            html.I("Developed by: Rita Miller, Data Scientist, RN"),
            html.Br(),
            "Date: ",
            html.Span(date.today().strftime("%B %d, %Y"))
        ])
    ],
    # background color for entire app
       style={'text-align':'center', 'font-size':'22px','background-color':'rgb(224, 255, 252)'})

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment


<IPython.core.display.Javascript object>

In [32]:
# Dash App for Frequency of RRTs in Current Month

# Group by date and RRT ONLY columns
rrt_frequency = df_cleaned.groupby([df_cleaned.columns[0], df_cleaned.columns[3]]).size().reset_index(name='Count')

# Sort by Date to ensure dates are in ascending order with total count in legend
rrt_frequency = rrt_frequency.sort_values(by=df_cleaned.columns[0], ascending=True)

# Rename columns for clarity
rrt_frequency.columns = ['Date', 'RRT Occurrences', 'Count']

# Calculate the total count
total_count = rrt_frequency['Count'].sum()

# Create the plot with plotly.express
fig = px.line(rrt_frequency, x='Date', y='Count', markers=True, title='September 2024: RRT Occurrences by Date')

# Add a trace for the total count in the legend
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(size=0),
    showlegend=True,
    name=f'Total Count: {total_count}'
))

# Update layout for better visibility
fig.update_layout(
    xaxis_title='Dates',
    yaxis_title='Frequency',
    xaxis=dict(tickangle=45)
)
# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Daily Frequency of RRTs'),

    html.Div(children=[
        dcc.Graph(
            id='line_graph',
            figure=fig
        )
    ], style={'border': '5px dotted green'})  # Add border style
])
app.layout.children.append(
    html.Div(children=[
        html.H1(children='Explore the Daily RRT Frequency for September 2024'),
        html.Div(children=[
            dcc.Dropdown(
                id='date-dropdown',
                options=[{'label': date, 'value': date} for date in rrt_frequency['Date']],
                value=rrt_frequency['Date'][0],
                style={'width': '200px', 'margin': '0 auto', 'float': 'left'}
            )
        ]),
        html.Div(id='output-container', style={'width': '350px', 'height': '50px', 'display': 'inline-block',
                                               'vertical-align': 'top', 'border': '5px dotted blue', 'padding': '20px'})
    ])
)
# Callback to update the output container
@app.callback(
    Output('output-container', 'children'),
    [Input('date-dropdown', 'value')]
)
def update_output(selected_date):
    try:
        frequency = rrt_frequency[rrt_frequency['Date'] == selected_date]['Count'].values[0]
        return f'The frequency of RRTs on {selected_date} was {frequency}.\nThank you for Exploring the Swedish First Hill ICU Dashboard!'
    except IndexError:
        return f'No data available for {selected_date}.\nThank you for Exploring the Swedish First Hill ICU Dashboard!'

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment

<IPython.core.display.Javascript object>

In [33]:
#Dash App for the Frequency of RRTs by Location (Top 12)

# Calculate the frequency of each location (top 12)
frequency_loc = df_cleaned[df_cleaned.columns[4]].value_counts().head(12).reset_index()
frequency_loc.columns = ['Location', 'Frequency']

# Create the bar plot with data labels
fig = px.bar(frequency_loc, x='Location', y='Frequency', text='Frequency', title='September 2024: Frequency of RRTs by Location (Top 12)')

# Automatically display the text on bars
fig.update_traces(textposition='outside')

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Frequency of RRTs by Location (Top 12)'),
    html.Div(children=[
        dcc.Graph(
            id='bar_graph',
            figure=fig
        )
    ], style={'border': '5px dotted green'})  # Add border style
])

# Define the layout
# Additional layout for dropdown and output container
app.layout.children.append(
    html.Div(children=[
    html.H1(children='Explore the Frequency of RRTs by Location (Top 12) for September 2024'),
    html.Div(children=[
        dcc.Dropdown(
            id='location-dropdown',
            options=[{'label': location, 'value': location} for location in frequency_loc['Location']],
            value=frequency_loc['Location'].iloc[0]
        ),
    ], style={'width': '200px', 'margin': '0 auto', 'float': 'left'}),  # Align dropdown to the left
    html.Div(id='output-container', style={'width': '350px', 'height': '50px', 'display': 'inline-block',
                                           'vertical-align': 'top', 'border': '5px dotted blue', 'padding': '20px'})
])
)

# Callback to update the output container
@app.callback(
    Output('output-container', 'children'),
    [Input('location-dropdown', 'value')]
)

def update_output(selected_location):
    try:
        frequency = frequency_loc[frequency_loc['Location'] == selected_location]['Frequency'].values[0]
        return f'The frequency of RRTs for {selected_location} was {frequency}.\nThank you for exploring the Swedish First Hill ICU Dashboard!'
    except IndexError:
        return f'No data available for {selected_location}.\nThank you for exploring the Swedish First Hill ICU Dashboard!'

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment


<IPython.core.display.Javascript object>

In [34]:
#Frequency of RRTs by Shifts

# Convert the timestamp column to datetime
#df_cleaned[df_cleaned.columns[1]] = pd.to_datetime(df_cleaned[df_cleaned.columns[1]])


# Convert the timestamp column to datetime
df_cleaned[1] = pd.to_datetime(df_cleaned[1], errors='coerce')

# Drop rows with NaT values if necessary
df_cleaned = df_cleaned.dropna(subset=[1])


# Define the bins and labels for the shifts
bins = [0, 7, 19, 24]
labels = ['Night Shift', 'Day Shift', 'Night Shift']

# Categorize the timestamps into shifts
df_cleaned['Shift'] = pd.cut(df_cleaned[df_cleaned.columns[1]].dt.hour, bins=bins, labels=labels,
                             right=False, include_lowest=True, ordered=False)

# Calculate the frequency by shift
frequency_shift = df_cleaned['Shift'].value_counts().reset_index()
frequency_shift.columns = ['Shift', 'Frequency']

# Create the bar plot with data labels
fig = px.bar(frequency_shift, x='Shift', y='Frequency', text='Frequency', title='September 2024: Frequency of RRTs by Shift')

# Automatically display the text on bars
fig.update_traces(textposition='outside')

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Frequency of RRTs by Shift'),
    html.Div(children=[
        dcc.Graph(
            id='bar_graph',
            figure=fig
)
    ], style={'border': '5px dotted green'})  # Add border style
])

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment

gc.enable()

<IPython.core.display.Javascript object>

In [35]:
#Frequency of RRTs by Triggers

# Frequency of RRT by Triggers in Current Month
# Calculate the frequency of each trigger
frequency_triggers = df_cleaned[df_cleaned.columns[2]].value_counts().reset_index()
frequency_triggers.columns = ['Triggers', 'Frequency']

# Create the bar plot with data labels
fig = px.bar(frequency_triggers, x='Triggers', y='Frequency', text='Frequency', title='September 2024: Frequency of RRTs by Triggers')

# Automatically display the text on bars
fig.update_traces(textposition='outside')

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Frequency of RRTs by Triggers'),
    html.Div(children=[
        dcc.Graph(
            id='bar_graph',
            figure=fig
        )
    ], style={'border': '5px dotted green'})  # Add border style
])

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment



<IPython.core.display.Javascript object>

In [36]:
# Frequency of RRTs by Higher Level of Care (HLC)

# Calculate the frequency of each Higher Level of Care (HLC)
frequency_hlc = df_cleaned[df_cleaned.columns[5]].value_counts().reset_index()
frequency_hlc.columns = ['HLC', 'Frequency']

# Create the bar plot with data labels
fig = px.bar(frequency_hlc, x='HLC', y='Frequency', text='Frequency', title='September 2024: Frequency of RRTs by Higher Level of Care (HLC)')

# Automatically display the text on bars
fig.update_traces(textposition='outside')

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Frequency of RRTs by Higher Level of Care (HLC)'),
    html.Div(children=[
        dcc.Graph(
            id='bar_graph',
            figure=fig
        )
    ], style={'border': '5px dotted green'})  # Add border style
])

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment


<IPython.core.display.Javascript object>

In [37]:
# Frequency of RRT by Disposition

frequency_outcome = df_cleaned[df_cleaned.columns[6]].value_counts().reset_index()
frequency_outcome.columns = ['Disposition', 'Frequency']

# Create the bar plot with data labels
fig = px.bar(frequency_outcome, x='Disposition', y='Frequency', text='Frequency',
             title='September 2024: Frequency of RRTs by Disposition')

# Automatically display the text on bars
fig.update_traces(textposition='outside')

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Add this line for deployment

# Define the layout
app.layout = html.Div(children=[
    html.H1(children='September 2024: Frequency of RRTs by Disposition'),
    html.Div(children=[
        dcc.Graph(
            id='bar_graph',
            figure=fig
        )
    ], style={'border': '5px dotted green'})  # Add border style
])

# Run the server/app
if __name__ == '__main__':
    app.run_server(debug=True, port=int(os.environ.get('PORT', 5000)))  # Ensure the port is set for deployment

gc.enable()

<IPython.core.display.Javascript object>